In [22]:
from cartes.osm import Overpass

tokyo_subway = Overpass.request(
    area={"name:en": "Tokyo", "admin_level": 4, "as_": "tokyo"},
    way=dict(railway='subway', area="tokyo")
)
tokyo_yamanote = Overpass.request(
    area={"name:en": "Tokyo", "admin_level": 4},
    way=dict(railway=True, name=dict(regex="山手線$")),
)


In [23]:
#tokyo_railway = tokyo_subway.data.query("type_ == 'way' and name == name") #['name:en'].unique()
#tokyo_railway = tokyo_railway.loc[:, tokyo_railway.columns != 'geometry']
#tokyo_railway.to_csv("./test.csv")

from shapely.ops import linemerge, split
from shapely.geometry import LineString, MultiLineString
import pandas as pd
import geopandas as gpd
import os
tokyo_railway = tokyo_subway.data.query("type_ == 'way' and name == name") #['name:en'].unique()
tokyo_railway = tokyo_railway[["name:en", 'colour',"geometry"]]

replacements = {"–": "-", "ō": "o", "ū": "u", "line": "Line"}

for key, value in replacements.items():
    tokyo_railway['name:en'] = tokyo_railway['name:en'].str.replace(key, value, regex=False)

tokyo_railway = tokyo_railway.groupby('name:en').agg({
    'geometry': lambda x: x.union_all(),  # Union all geometries
    'colour': 'first'  # Take the first colour (assuming same colour within group)
}).reset_index()
tokyo_railway['colour'] = tokyo_railway['colour'].fillna("#000000")
tokyo_railway = gpd.GeoDataFrame(tokyo_railway, geometry='geometry')


def crop_line_at_longitude(geom, longitude=139.5):
    vertical_line = LineString([(longitude, -90), (longitude, 90)])
    split_geom = split(geom, vertical_line)
    cropped_geoms = [part for part in split_geom.geoms if any(point[0] > longitude for point in part.coords)]
    
    #Return the cropped geometry, if it exists
    if cropped_geoms:
        return MultiLineString(cropped_geoms)  # Or combine them if you need multiple parts
    else:
        return None 

tokyo_railway['geometry'] = tokyo_railway['geometry'].apply(crop_line_at_longitude)



drop_list = ["Arrival", "Container", "Engine", "Storage", "Freight", "Repair", "Pass", "Pre-departure", "Wheel", "SKY ACCESS", "Lead"]
tokyo_railway = tokyo_railway[~tokyo_railway['name:en'].str.contains("|".join(drop_list), case=False, na=False)].reset_index()
tokyo_railway['color_int'] = tokyo_railway['colour'].replace("DeepSkyBlue", "#00bfff").apply(lambda x: int(x[1:], 16))
tokyo_railway['GlobalID'] = tokyo_railway['name:en']
tokyo_railway.plot(color=tokyo_railway['colour'])
tokyo_railway.loc[:, tokyo_railway.columns != 'geometry'].to_csv("./test.csv")
os.remove("./test3.geojson")
tokyo_railway.to_file("./test3.geojson")

In [24]:
from shapely.ops import linemerge
from shapely import MultiLineString, LineString, Point
import pandas as pd
import geopandas as gpd
import os

lines = tokyo_subway.data.query("type_ == 'way' and name == name").assign(
    name=lambda df: df.name.str.split(" ", n=1, expand=True)[0]
)

yamanote = tokyo_yamanote.data.query("type_ == 'way' and name == name").assign(
    name=lambda df: df.name.str.split(" ", n=1, expand=True)[0]
)

lines = pd.concat([lines, yamanote])


lines.loc[lines["name:en"] == "Toei Mita Line", "name"] = "都営地下鉄三田線"
lines.loc[lines["name:en"] == "Toei Asakusa Line", "name"] = "都営地下鉄浅草線"
lines.loc[lines["name"] == "京王電鉄京王線", "name:en"] = "Tokyo Metro Marunouchi Line"
lines.loc[lines["name"] == "東京メトロ丸ノ内線", "name:en"] = "Tokyo Metro Marunouchi Line"
lines = lines[lines["name:en"] != "Tokyo Metro"]

def points_are_close(point1, point2, tolerance=1e-6):
    return point1.distance(point2) <= tolerance

def sort_and_combine_lines(lines, tolerance=1e-6):
    ordered_lines = [lines.pop(0)]
    
    while lines:
        current_line = ordered_lines[-1]
        current_end = Point(current_line.coords[-1])
        found_connection = False
        
        for i, line in enumerate(lines):
            start = Point(line.coords[0])
            end = Point(line.coords[-1])
            
            if points_are_close(current_end, start, tolerance):
                ordered_lines.append(line)
                lines.pop(i)
                found_connection = True
                break
            elif points_are_close(current_end, end, tolerance):
                ordered_lines.append(LineString(line.coords[::-1]))  # Reverse the line
                lines.pop(i)
                found_connection = True
                break
        
        # If no connection was found, start a new segment
        if not found_connection:
            ordered_lines.append(lines.pop(0))
    
    return MultiLineString(ordered_lines)

def merge_line(elt):
    try:
        name_en = elt["name:en"].max()
    except TypeError:
        name_en = "Unk"
    return pd.Series(
        {
            "geometry": sort_and_combine_lines(elt.geometry.tolist()),
            "name:en": name_en
        }
    )
    
lines = (
    lines[["name", "name:en", "geometry"]]
    .groupby("name").apply(merge_line).reset_index()
)

lines.loc[:, lines.columns != 'geometry'].to_csv("./test.csv")
lines = gpd.GeoDataFrame(lines, geometry="geometry")
os.remove("./test.geojson")
lines.to_file("./test.geojson")
#lines.head()

#type(lines['name:en'][0].max())

In [25]:
import os
tokyo_yamanote = Overpass.request(
    area={"name:en": "Tokyo", "admin_level": 4},
    way=dict(railway=True, name=dict(regex="山手線$")),
)
#The geometry will be enough here
yamanote = tokyo_yamanote.data
yamanote = (
    yamanote[["name", "name:en", "geometry"]]
    .groupby("name").apply(merge_line).reset_index()
)
os.remove("./test_2.geojson")
yamanote.to_file("./test_2.geojson")
#tokyo_yamanote.data.to_file("./test_2.geojson")